# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [219]:
from abc import ABC, abstractmethod
from collections.abc import Iterable, Mapping
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, \
    PegasusTokenizerFast, pipeline

from htools import *
from incendio.utils import DEVICE, gpu_setup

In [4]:
gpu_setup(False)

cpu


/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:38: UserWarning: Cuda not available.
  if not torch.cuda.is_available(): warnings.warn('Cuda not available.')
/Users/hmamin/anaconda3/lib/python3.7/site-packages/incendio/utils.py:41: UserWarning: Incendio device is not cuda.
  warnings.warn('Incendio device is not cuda.')


In [2]:
version = 'tuner007/pegasus_paraphrase'
net = PegasusForConditionalGeneration.from_pretrained(version)
tok = PegasusTokenizerFast.from_pretrained(version)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60954), raddr=('52.216.233.221', 443)>
  self._sock = None
/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60955), raddr=('52.216.233.221', 443)>
  self._sock = None


/Users/hmamin/anaconda3/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:69: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/hmamin/.cache/torch/transformers/fa4532c0035b101d7abcd5c0c9c34a83288902b66c5616034db1a47643e05c75.efce77b8dcd2c57b109b0d10170fcdcd53f23c21286974d4f66706536758ab6e'>
  m.ParseFromString(open(filename, "rb").read())


In [39]:
_ = net.to(DEVICE)

In [50]:
texts = [
    'Educational games and digital learning materials to provide K-12 '
    'students with enriching experiences.',
    'The world\'s largest social network. Helping people build and maintain '
    'relationships in a disconnected world.',
    'I hate school. I wish my teacher would leave me alone. I don\'t think '
    ' he likes me.',
    'Today the president announced new plans to revamp the private '
    'healthcare system. Pundits questioned how he would manage to pass the '
    'bill.'
]

In [5]:
hdir(tok)

Using bos_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.


{'SPECIAL_TOKENS_ATTRIBUTES': 'attribute',
 'add_special_tokens': 'method',
 'add_tokens': 'method',
 'additional_special_tokens': 'attribute',
 'additional_special_tokens_ids': 'attribute',
 'all_special_ids': 'attribute',
 'all_special_tokens': 'attribute',
 'all_special_tokens_extended': 'attribute',
 'backend_tokenizer': 'attribute',
 'batch_decode': 'method',
 'batch_encode_plus': 'method',
 'bos_token': 'attribute',
 'bos_token_id': 'attribute',
 'build_inputs_with_special_tokens': 'method',
 'clean_up_tokenization': 'method',
 'cls_token': 'attribute',
 'cls_token_id': 'attribute',
 'convert_ids_to_tokens': 'method',
 'convert_tokens_to_ids': 'method',
 'create_token_type_ids_from_sequences': 'method',
 'decode': 'method',
 'deprecation_warnings': 'attribute',
 'encode': 'method',
 'encode_plus': 'method',
 'eos_token': 'attribute',
 'eos_token_id': 'attribute',
 'from_pretrained': 'method',
 'get_added_vocab': 'method',
 'get_special_tokens_mask': 'method',
 'get_vocab': 'metho

In [6]:
res = tok.prepare_seq2seq_batch(texts, truncation=True, padding='longest')

In [7]:
res

{'input_ids': tensor([[11263,   727,   111,  1016,   761,   917,   112,   319,  1046,  6054,
           392,   122, 26838,  1747,   107,     1,     0,     0,     0,     0,
             0,     0,     0],
        [  139,   278,   131,   116,  1368,   525,   952,   107, 22844,   200,
           736,   111,  1634,  2074,   115,   114, 20402,   278,   107,     1,
             0,     0,     0],
        [  125,  4180,   399,   107,   125,  1216,   161,  2118,   192,   858,
           213,  1600,   107,   125,   272,   131,   144,   311,   178,  5606,
           213,   107,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
gen = net.generate(**res)

tensor([[   0, 1046, 6054,  392,  137,  207, 2387,  727,  111, 1016,  761,  917,
          107,    1,    0,    0,    0],
        [   0,  139,  278,  131,  116, 1368,  525,  952,  107,    1,    0,    0,
            0,    0,    0,    0,    0],
        [   0,  125,  272,  131,  144,  172,  399,  111,  161, 2118,  591,  131,
          144,  172,  213,  107,    1]])

In [19]:
tok.batch_decode(gen.tolist(), skip_special_tokens=True)

['K-12 students can use educational games and digital learning materials.',
 "The world's largest social network.",
 "I don't like school and my teacher doesn't like me."]

In [54]:
@add_docstring(net.generate)
def paraphrase(text, n=1, temperature=1.5, **gen_kwargs):
    batch = tok.prepare_seq2seq_batch([text], truncation=True, 
                                      padding='longest').to(DEVICE)
    paraphrased = net.generate(**batch, num_return_sequences=n,
                               temperature=temperature, **gen_kwargs)
    return tok.batch_decode(paraphrased.tolist(), skip_special_tokens=True)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
paraphrase(texts[3], n=5, num_beams=10)

['The president announced plans to reform the private healthcare system.',
 'Pundits wondered how the president would pass the bill.',
 'Pundits questioned how the president would pass the bill.',
 'Pundits were questioning how the president would pass the bill.',
 'Pundits questioned how he would get the bill passed.']

In [53]:
texts[-1]

'Today the president announced new plans to revamp the private healthcare system. Pundits questioned how he would manage to pass the bill.'

In [48]:
net.generate()

{'input_ids': <Parameter "input_ids: Union[torch.LongTensor, NoneType] = None">,
 'decoder_input_ids': <Parameter "decoder_input_ids: Union[torch.LongTensor, NoneType] = None">,
 'max_length': <Parameter "max_length: Union[int, NoneType] = None">,
 'min_length': <Parameter "min_length: Union[int, NoneType] = None">,
 'do_sample': <Parameter "do_sample: Union[bool, NoneType] = None">,
 'early_stopping': <Parameter "early_stopping: Union[bool, NoneType] = None">,
 'num_beams': <Parameter "num_beams: Union[int, NoneType] = None">,
 'temperature': <Parameter "temperature: Union[float, NoneType] = None">,
 'top_k': <Parameter "top_k: Union[int, NoneType] = None">,
 'top_p': <Parameter "top_p: Union[float, NoneType] = None">,
 'repetition_penalty': <Parameter "repetition_penalty: Union[float, NoneType] = None">,
 'bad_words_ids': <Parameter "bad_words_ids: Union[Iterable[int], NoneType] = None">,
 'bos_token_id': <Parameter "bos_token_id: Union[int, NoneType] = None">,
 'pad_token_id': <Para

In [58]:
class ParaphrasePipeline:
    
    name = 'tuner007/pegasus_paraphrase'
    
    def __init__(self, net=None, tok=None):
        self.net = net or PegasusForConditionalGeneration.from_pretrained(
            self.name).to(DEVICE)
        self.tok = tok or PegasusTokenizerFast.from_pretrained(self.name)
        
    @add_docstring(PegasusForConditionalGeneration.generate)
    def __call__(self, text, n=1, temperature=1.5, **kwargs):
        # TODO: not sure how many rows of text can be done in a single batch.
        # Maybe look at other pipelines to see what they do. I'm thinking we
        # could auto-batch longer sequences (e.g. a list w/ 10_000 strings
        # might become 100 batches of 100).
        texts = [text] if isinstance(text, str) else text
        batch = self.tok.prepare_seq2seq_batch(texts, truncation=True, 
                                               padding='longest').to(DEVICE)
        gen_tokens = self.net.generate(**batch, num_return_sequences=n,
                                       temperature=temperature, **kwargs)
        gen = self.tok.batch_decode(gen_tokens.tolist(), 
                                    skip_special_tokens=True)
        if not isinstance(text, str) and len(text) > 1: 
            gen = [gen[i*n:(i+1)*n] for i in range(len(text))]
        return gen

In [301]:
p_pipe = ParaphrasePipeline(net, tok)

In [302]:
p_pipe('It was a beautiful rainy day.')

['There was a beautiful day.']

In [303]:
p_pipe('It was a beautiful rainy day.', n=3)

['There was a beautiful day.',
 'It was a nice day.',
 'It was raining but it was nice.']

In [54]:
p_pipe(['It was a beautiful rainy day.', 'The duck was yellow and fluffy.'])

[['There was a beautiful day.'], ['The duck was fluffy and yellow.']]

In [55]:
p_pipe(['It was a beautiful rainy day.', 'The duck was yellow and fluffy.'], 
     n=3)

[['There was a beautiful day.',
  'It was a nice day.',
  'It was raining but it was nice.'],
 ['The duck was fluffy and yellow.',
  'The duck was fluffy.',
  'The duck was big and fluffy.']]

In [56]:
p_pipe(['It was a beautiful rainy day.'], n=2)

['There was a beautiful day.', 'It was a nice day.']

In [274]:
# Decided to abandon this. Too many ways children differ: paraphrase pipeline
# can't create pipe with pipeline(name) because it's not part of huggingface,
# paraphrase transform doesn't need to check if listlike because preprocess
# does nothing and __call__ literally just calls pipe. Leaving this here as
# an examle of init_subclass.
class TransformerTransformBase(ABC):
    
    def __init__(self):
        pass
    
    def __call__(self, text, **kwargs):
        if listlike(text):
            return [self.transform(t, **kwargs) for t in text]
        return self.transform(text, **kwargs)
    
    @abstractmethod
    def transform(self, text, **kwargs):
        pass
    
    def __init_subclass__(cls, **kwargs):
        if not hasattr(cls, 'pipe_name'):
            raise RuntimeError(f'{cls} must have class attr "pipe_name".')

In [275]:
class Tmp(TransformerTransformBase):
    """a"""

RuntimeError: <class '__main__.Tmp'> must have class attr "pipe_name".

In [993]:
@auto_repr
class FillMaskTransform:    

    MASK = '<mask>'
    
    def __init__(self, pipe=None, n=3):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or pipeline('fill-mask', topk=n)
        self.n = n
        
        assert type(self.pipe).__name__ == 'FillMaskPipeline'
    
    def _preprocess(self, text, min_keep=3, errors='raise'):
        """
        errors: str
            If 'warn', we show a warning when min_keep is violated but allow
            masking to take place.
            """
        if listlike(text):
            return [self._preprocess(row, min_keep, errors) for row in text]
        
        tokens = text.split()
        if len(tokens) < min_keep + 1:
            msg = (f'Text "{text[:25]}..." is too short to mask while '
                   f'enforcing min_keep={min_keep}.')
            if errors == 'warn':
                warnings.wazzaarn(msg)
            else:
                raise ValueError(msg)
        
        idx = np.random.choice(range(len(tokens)))
        return ' '.join(self.MASK if i == idx else t 
                        for i, t in enumerate(tokens))
    
    def __call__(self, text, n=1, n_mask=1, min_keep=3, return_all=False, 
                 errors:('warn', 'raise')='raise', 
                 strategy:('random', 'best')='best'):
        """
        n: int or None
            If None, return all generated examples for the given mask count.
            This can become very large when n_mask is large. Recall pipeline
            can only fill a single mask at a time. e.g. if self.n is
            3, n=None, and n_mask is 4, we first mask once and generate 3
            samples. Then we mask each of those 3 and generate a total of 9 
            samples, then 27, then finally 81 which is what will be returned.
            The intermediate samples can be returned with `return_all=True`.
        """
        # Each item will be a list of strings. Each string in res[i]
        # will have i words changed. If text is a sequence of strings, we must
        # handle each one separately because each is passed through pipeline
        # repeatedly.
        if listlike(text):
            return [self(row, n, n_mask, min_keep, return_all, errors) 
                    for row in text]
        
        # topk determines number of sequences generated by each model call.
        if n and n > self.pipe.topk:
            self.n = n
            
        res = [[text]]
        for i in range(n_mask):
            seqs = self.pipe(self._preprocess(res[-1], min_keep=min_keep,
                                              errors=errors))
            # Transformers returns either list of dicts or list of list of 
            # dicts depending on whether input list has 1 item or multiple.
            if isinstance(seqs[0], list): 
                seqs = [seq for group in seqs for seq in group]
            text = [seq['sequence'].replace('<s>', '').replace('</s>', '') 
                    for seq in seqs]
            
            # n=None selects all items when list slicing but only 1 in
            # `random.choice` so we check for it explicitly.
            if n and strategy == 'random':
                text = np.random.choice(text, n, replace=False)
            elif strategy == 'best':
                text = text[:n]
            res.append(text)
        if not return_all: res = res[n_mask]
        return res
    
    @property
    def n(self):
        return self._n
    
    @n.setter
    def n(self, n):
        if not isinstance(n, int):
            raise TypeError('n must be an integer.')
        
        self._n = n
        self.pipe.topk = n

In [994]:
t = 'I went to the store today to buy eggs.'
ts = [t, 'The bird swooped down onto the picnic table and squawked loudly.']

In [995]:
# m_pipe = pipeline('fill-mask')
m_tfm = FillMaskTransform(m_pipe)
m_tfm

FillMaskTransform(pipe=<transformers.pipelines.FillMaskPipeline object at 0x1a31ecfb70>)

In [996]:
m_tfm._preprocess(t)

'I went to the store today to <mask> eggs.'

In [997]:
m_tfm._preprocess(ts)

['I went to the store today to <mask> eggs.',
 'The bird swooped down onto <mask> picnic table and squawked loudly.']

In [998]:
m_tfm(t, 3, return_all=True)

[['I went to the store today to buy eggs.'],
 ['I went to the store yesterday to buy eggs.',
  'I went to the store intending to buy eggs.',
  'I went to the store wanting to buy eggs.']]

In [999]:
m_tfm(t, 3, n_mask=2, strategy='best')

['I went to another supermarket today to buy eggs.',
 'I went to a supermarket today to buy eggs.',
 'I went to my supermarket today to buy eggs.']

In [1000]:
m_tfm(t, 3, n_mask=2, strategy='random')

array(['I went to the store myself & buy eggs.',
       'I went to the store yesterday & buy eggs.',
       'I went to grocery store today and buy eggs.'], dtype='<U43')

In [1001]:
m_tfm(t, 3, n_mask=2, return_all=True)

[['I went to the store today to buy eggs.'],
 ['I went to the store today to buy eggs.',
  'We went to the store today to buy eggs.',
  'She went to the store today to buy eggs.'],
 ['I went to the store today to buy eggs.',
  'We went to the store today to buy eggs.',
  'She went to the store today to buy eggs.']]

In [1002]:
m_tfm(t, 1, return_all=True)

[['I went to the store today to buy eggs.'],
 ['I went to the store today to buy eggs.']]

In [1003]:
m_tfm(ts)

[['I went to the supermarket today to buy eggs.'],
 ['The bird swooped down onto the picnic table and squawked loudly.']]

In [1004]:
m_tfm(ts, 2)

[['I went to grocery store today to buy eggs.',
  'I went to the store today to buy eggs.'],
 ['The bird swooped down onto the picnic table and squawked loudly.',
  'The bird swooped down onto the kitchen table and squawked loudly.']]

In [1005]:
m_tfm(ts, n=None, n_mask=2)

[['I went to the supermarket today to buy eggs.',
  'I drove to the supermarket today to buy eggs.',
  'I headed to the supermarket today to buy eggs.',
  'I went to the store today to buy eggs.',
  'I went into the store today to buy eggs.',
  'I went through the store today to buy eggs.',
  'I went to the grocery today to buy eggs.',
  'She went to the grocery today to buy eggs.',
  'We went to the grocery today to buy eggs.'],
 ['The bird swooped down onto the picnic table and squawked loudly.',
  'The bird swooped up onto the picnic table and squawked loudly.',
  'The bird swooped overhead onto the picnic table and squawked loudly.',
  'The bird swooped down onto the kitchen table and squawked loudly.',
  'The bird swooped down onto the kitchen floor and squawked loudly.',
  'The bird swooped down onto the kitchen counter and squawked loudly.',
  'The bird hopped down onto the dining table and squawked loudly.',
  'The bird slid down onto the dining table and squawked loudly.',
  '

In [1006]:
m_tfm(ts, n=None, n_mask=2, return_all=True)

[[['I went to the store today to buy eggs.'],
  ['I went to grocery store today to buy eggs.',
   'I went to the store today to buy eggs.',
   'I went to my store today to buy eggs.'],
  ['I went to grocery store today to buy eggs.',
   'I went to the store today to buy eggs.',
   'I went to my store today to buy eggs.',
   'I went to grocery store today to buy eggs.',
   'I went to the store today to buy eggs.',
   'I went to my store today to buy eggs.',
   'I went to my store today to buy eggs.',
   'I went to my supermarket today to buy eggs.',
   'I went to my grocer today to buy eggs.']],
 [['The bird swooped down onto the picnic table and squawked loudly.'],
  ['The bird swooped down onto a picnic table and squawked loudly.',
   'The bird swooped down onto the picnic table and squawked loudly.',
   'The bird swooped down onto another picnic table and squawked loudly.'],
  ['The bird hopped down onto a picnic table and squawked loudly.',
   'The bird settled down onto a picnic ta

In [519]:
@auto_repr
class ParaphraseTransform:
    """Not sure how useful this will really be but this basically just 
    wraps ParaphrasePipeline to share a more similar interface with the other
    NLP transforms.
    """

    def __init__(self, pipe=None, n=1):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or ParaphrasePipeline()
        self.n = n
            
        assert type(self.pipe).__name__ == 'ParaphrasePipeline'
        
    def _preprocess(self, text):
        return text
    
    def __call__(self, text, n=None, **kwargs):
        return self.pipe(text, n=n or self.n, **kwargs)

In [304]:
p_tfm = ParaphraseTransform(p_pipe)
p_tfm

ParaphraseTransform(pipe=<__main__.ParaphrasePipeline object at 0x1a47cda198>)

In [305]:
p_tfm('The beach is loud and crowded today.', n=3)

['The beach is crowded.',
 'There is a lot of people on the beach.',
 'There is a lot of people at the beach.']

In [174]:
p_tfm(ts)

[['I went to buy eggs.'], ['The bird swooped down onto the picnic table.']]

In [175]:
p_tfm(ts, n=2)

[['I went to buy eggs.', 'I went to the store to buy eggs.'],
 ['The bird swooped down onto the picnic table.',
  'The bird swooped down on the picnic table.']]

In [227]:
def listlike(x):
    """Checks if an object is a list/tuple/set/array etc. Strings and
    mappings (e.g. dicts) are not considered list-like.
    """
    return isinstance(x, Iterable) and not isinstance(x, (str, Mapping))

In [226]:
for obj in ('a', 6, [], (), {}, set(), [3, 4], ['a', 'b'], ('a',), {'a': 'b'},
            {'a', 'b'}, np.arange(5)):
    print(type(obj), listlike(obj))

<class 'str'> False
<class 'int'> False
<class 'list'> True
<class 'tuple'> True
<class 'dict'> False
<class 'set'> True
<class 'list'> True
<class 'list'> True
<class 'tuple'> True
<class 'dict'> False
<class 'set'> True
<class 'numpy.ndarray'> True


In [534]:
@auto_repr
class GenerativeTransform:
    
    def __init__(self, pipe=None, n=1):
        # We let user pass in pipe at least for now since re-instantiating the
        # class can be very slow during development. Need to consider whether
        # I want this behavior to remain.
        self.pipe = pipe or pipeline('text-generation')
        self.n = n
        
        assert type(self.pipe).__name__ == 'TextGenerationPipeline'
    
    def _preprocess(self, text, drop=None, drop_pct=None, rand_low=None, 
                    rand_high=None, min_keep=3, return_tuple=False):
        """Truncate text."""
        if listlike(text):
            return [self._preprocess(row, drop, drop_pct, rand_low, rand_high,
                                     min_keep, return_tuple) for row in text]
        
        tokens = text.split()
        if len(tokens) <= min_keep:
            n = 0
        else:
            # Default is to truncate the last 20% of the sequence.
            if drop:
                n = drop
            elif drop_pct:
                n = int(drop_pct * len(tokens))
            elif rand_low is not None and rand_high is not None:
                n = np.random.randint(rand_low, rand_high)
            else:
                n = int(np.ceil(.2 * len(tokens)))
            n = np.clip(n, 0, len(tokens) - min_keep)
            tokens = tokens[:-n]
        truncated = ' '.join(tokens)
        return (truncated, n) if return_tuple else truncated
    
    def __call__(self, text, n=None, min_length=2, max_length=7, 
                 **generate_kwargs):
        n = n or self.n
        if listlike(text):
            return [self(row, n, min_length, max_length, **generate_kwargs) 
                    for row in text]
    
        # `generate` counts current length as part of min_length. 
        text = self._preprocess(text)
        n_curr = len(self.pipe.tokenizer.tokenize(text))
        res = self.pipe(text, min_length=n_curr + min_length,
                        max_length=n_curr + max_length,
                        num_return_sequences=n, **generate_kwargs)
        return [row['generated_text'] for row in res]

In [535]:
# g_pipe = pipeline('text-generation')
g_tfm = GenerativeTransform(g_pipe)
g_tfm

GenerativeTransform(pipe=<transformers.pipelines.TextGenerationPipeline object at 0x1d081425c0>, n=1)

In [536]:
g_tfm._preprocess(t)

'I went to the store today to'

In [537]:
g_tfm._preprocess(ts, return_tuple=True)

[('I went to the store today to', 2),
 ('The bird swooped down onto the picnic table', 3)]

In [538]:
g_tfm(t)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['I went to the store today to buy a shirt, but I felt']

In [539]:
g_tfm(t, n=2)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['I went to the store today to buy some clothes. The first thing',
 'I went to the store today to see where it went and I asked']

In [540]:
g_tfm(ts)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[['I went to the store today to check out everything. I wanted to'],
 ['The bird swooped down onto the picnic table so much that two of them noticed']]

In [541]:
g_tfm(ts, n=2)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[['I went to the store today to pick up my kids, got some',
  "I went to the store today to check out his new book, '"],
 ['The bird swooped down onto the picnic table, then proceeded to lay itself down',
  'The bird swooped down onto the picnic table, where he and the other dogs']]

In [285]:
# I'm now thinking this isn't that useful. It also loses our kwarg names 
# unless I pull some tricks altering signatures. Don't think this offers 
# enough benefit to justify that.
class TransformerTransform:
    
    def __init__(self, mode, pipe=None):
        self.mode = mode
        self._transformer = self._get_transformer(pipe)
        
    def _preprocess(self, text, **kwargs):
        return self._transformer._preprocess(text, **kwargs)
    
    def __call__(self, text, **kwargs):
        return self._transformer(text, **kwargs)
    
    def _get_transformer(self, pipe):
        if self.mode == 'mask':
            return FillMaskTransform(pipe)
        elif self.mode == 'generate':
            return GenerativeTransform(pipe)
        elif self.mode == 'paraphrase':
            return ParaphraseTransform(pipe)
        else:
            raise ValueError('mode must be in (mask, generate, paraphrase).')

In [287]:
mt = TransformerTransform('mask', m_pipe)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [288]:
mt._preprocess(t)

'I went to <mask> store today to buy eggs.'

In [289]:
mt._preprocess(ts, n=2)

['I <mask> to the store today to <mask> eggs.',
 'The <mask> swooped down <mask> the picnic table and squawked loudly.']

In [290]:
mt(t)

['I went to the store today to buy eggs.',
 'I went into the store today to buy eggs.',
 'I went through the store today to buy eggs.',
 'I went around the store today to buy eggs.',
 'I went in the store today to buy eggs.']

In [291]:
mt(ts)

[['I went to the store today to buy eggs.',
  'I went into the store today to buy eggs.',
  'I went through the store today to buy eggs.',
  'I went around the store today to buy eggs.',
  'I went in the store today to buy eggs.'],
 ['The bird swooped down onto the picnic table and squawked loudly.',
  'The bird swooped down onto the picnic table, squawked loudly.',
  'The bird swooped down onto the picnic table then squawked loudly.',
  'The bird swooped down onto the picnic table who squawked loudly.',
  'The bird swooped down onto the picnic table but squawked loudly.']]

## Test on data

In [351]:
lines = [line.strip() for line 
         in '\n'.join(load(f'/Users/hmamin/data/bbc/tech/{n:03}.txt') 
         for n in range(1, 402)).split('.') if line]
len(lines)

9887

In [544]:
m_res = m_tfm(lines[:100], errors='warn')

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Text "And vice versa..." is too short to mask while enforcing min_keep=3.


In [549]:
len(flatten(m_res))

500

In [545]:
m_res[:2]

[["Ink helps drive democracy in Asia The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and fingerprint readers in the country's elections as part of a drive to prevent multiple voting",
  "Ink helps drive democracy in Asia The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and electronic readers in the country's elections as part of a drive to prevent multiple voting",
  "Ink helps drive democracy in Asia The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and pen readers in the country's elections as part of a drive to prevent multiple voting",
  "Ink helps drive democracy in Asia The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and paper readers in the country's elections as part of a drive to prevent multiple voting",
  "Ink helps drive democracy in Asia The Kyrgyz Republic, a sma

In [543]:
g_res = g_tfm(lines[:100])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad

In [548]:
len(flatten(g_res))

100

In [550]:
lines[:2]

["Ink helps drive democracy in Asia\n\nThe Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting",
 'This new technology is causing both worries and guarded optimism among different sectors of the population']

In [546]:
g_res[:2]

[["Ink helps drive democracy in Asia The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as an early warning to the rest of"],
 ['This new technology is causing both worries and guarded optimism among different groups in Europe about cybersecurity," said']]

In [558]:
p_res = p_tfm(lines[:25])

In [559]:
len(flatten(p_res))

25

In [560]:
lines[:5]

["Ink helps drive democracy in Asia\n\nThe Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting",
 'This new technology is causing both worries and guarded optimism among different sectors of the population',
 'In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections',
 'The US government agreed to fund all expenses associated with this decision',
 'The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies']

In [561]:
p_res[:5]

[["The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting."],
 ['There are both worries and guarded optimism among different sectors of the population.'],
 ['The law requiring the use of ink during the upcoming Parliamentary and Presidential elections was pushed through by the President in an effort to live up to its reputation as an island of democracy.'],
 ['All expenses associated with this decision will be funded by the US government.'],
 ['The high point of the Kyrgyz Republic was in the mid 1990s when it had a legislative branch with 75 deputies and was seen by many experts as backsliding.']]

## TODO: maybe see about adding auto GPU support for ParaphrasePipeline